In [2]:
# import our libraries
import requests
import pandas as pd
from bs4 import BeautifulSoup
import sys, json
import os
from pathlib import Path
import pandas as pd

In [4]:
#ticks = pd.read_csv(r'/Users/eviekiwi/CS506Spring2021Repository/10_K_Risk_Evaluator/data/actual.csv')
ticks = open('/Users/eviekiwi/CS506Spring2021Repository/10_K_Risk_Evaluator/data/actual.csv', 'r')
all_ticks = ticks.read().split('\n')
tickers = all_ticks[1:-1]
print(len(tickers))

234


In [5]:
local_location = '/Users/eviekiwi/Documents/2021Spring/CS506/project/webscrape/sec-edgar-filings'
base_url = r"https://www.sec.gov/Archives/edgar/data/"
full_path = Path(local_location)

In [313]:
full_path

PosixPath('/Users/eviekiwi/Documents/2021Spring/CS506/project/webscrape/sec-edgar-filings')

In [8]:
#tickers = ['ABUS', 'ACAD', 'ADAP', 'ADPT', 'ADVM', 'AERI','AGIO','AGLE','AKBA','AKRO']

all_txt = []
for t in tickers: 
    company = local_location+'/'+ t
    path = Path(company)
    txt_files = list(path.glob('**/*.txt'))
    all_txt.append(txt_files)
    
    
file_url = []

for i in range(len(all_txt)):
    f = all_txt[i][0]
    file = open(str(f), 'r').readlines()
    for line in file: 
        line = line.strip().split('\t')
        if 'ACCESSION NUMBER:' in line: 
            accession_num = line[2]
            accession_num_rmv = accession_num.replace('-','')
        elif 'CENTRAL INDEX KEY:' in line: 
            central_key = line[3]
        url = base_url + central_key + '/' + accession_num_rmv + '/' + accession_num + '-index-headers.html'
    file_url.append(url)

In [9]:
print(file_url[19])
print(len(file_url))

https://www.sec.gov/Archives/edgar/data/0001297184/000129718419000013/0001297184-19-000013-index-headers.html
234


In [337]:
#https://www.sec.gov/Archives/edgar/data/0001621227/000155837020001571/R4.htm
#https://www.sec.gov/Archives/edgar/data/0001337553/000162828020002027/R4.htm
print(len(file_url))

234


### SCRAPE REVENUE

In [339]:
temp = []
for test in range(len(file_url)): 
#     print(file_url[test])
    string = file_url[test].split('/')
    url = string[0] + '//' + string[2] + '/' + string[3] + '/' + string[4] + '/' + string[5] + '/' + string[6] + '/'+ string[7] + '/'+ 'R4.htm'
    cont = requests.get(url).content
    
    res = BeautifulSoup(cont, 'html.parser')
    t = res.text.split('\n')
    try:
        company = tickers[test]
        matching = [s for s in t if "$" in s]
        rev1 = matching[1]
        rev2 = matching[2]
        rev1 = int(matching[1].split(' ')[1].replace(',',''))*1000
        rev2 = int(matching[2].split(' ')[1].replace(',',''))*1000
        temp.append([rev1,rev2, company])
        
    except:
        pass

In [355]:
delta = []
for i in temp:
    if i[0] == 0 or i[1] == 0:
        delta.append('NA')
    elif i[0] > i[1]: 
        delta.append(1)
    else:
        delta.append(0)


[1, 1, 0, 1, 0, 1, 'NA', 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 'NA', 1, 1, 1, 1, 1, 0, 1, 1, 'NA', 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 'NA', 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 'NA', 0, 1, 1, 0, 'NA', 1, 1, 1, 0, 1, 'NA', 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 'NA', 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 'NA', 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 'NA', 1, 0, 1, 'NA']


In [356]:
print(len(delta))

177


In [357]:
new_rev = []
ticks = []
old_rev = []
for i in temp: 
    new = i[0]
    new_rev.append(new)
    old = i[1]
    old_rev.append(old)
    company = i[2]
    ticks.append(company)

In [358]:
df = pd.DataFrame({"company":ticks,
                         "new_revenue":new_rev,
                  "old_revenue":old_rev,
                  "delta": delta})


In [359]:
df

,company,new_revenue,old_revenue,delta
0,ABUS,6011000,5945000,1
1,ACAD,223807000,124901000,1
2,ADAP,1122000,59505000,0
3,ADPT,85071000,55663000,1
4,ADVM,250000000,1612000000,0
...,...,...,...,...
172,XERS,1627000,0,NA
173,XLRN,73993000,13991000,1
174,XNCR,11564000,29039000,0
175,YMAB,63492000,34269000,1


In [362]:
df.to_csv('revenue.csv', index=False)

In [363]:
test = pd.read_csv('revenue.csv')
test

,company,new_revenue,old_revenue,delta
0,ABUS,6011000,5945000,1.0
1,ACAD,223807000,124901000,1.0
2,ADAP,1122000,59505000,0.0
3,ADPT,85071000,55663000,1.0
4,ADVM,250000000,1612000000,0.0
...,...,...,...,...
172,XERS,1627000,0,NaN
173,XLRN,73993000,13991000,1.0
174,XNCR,11564000,29039000,0.0
175,YMAB,63492000,34269000,1.0


# Grab XML SUM

In [10]:
xml_all = []

for url in file_url: 
    normal_url = r"%s"%url
    test = normal_url.strip().split('/')
    documents_url = test[0] + '//' + test[2] + '/' + test[3] + '/' + test[4] + '/' + test[5] + '/' + test[6] + '/' + test[7] + '/' + 'index.json'
    #print(documents_url)
    documents_url = r"%s"%documents_url
    content = requests.get(documents_url).json()

    

    # for file in range(len(content['directory']['item'])):
    #     print(file)
    for file in content['directory']['item']:

    #Grab the filing summary and create a new url leading to the file so we can download it.

            if file['name'] == 'FilingSummary.xml':
                xml_summary = 'https://www.sec.gov' + content['directory']['name'] + "/" + file['name']

                xml_all.append(xml_summary)

                print('-' * 100)
                print('File Name: ' + file['name'])
                print('File Path: ' + xml_summary)


----------------------------------------------------------------------------------------------------
File Name: FilingSummary.xml
File Path: https://www.sec.gov/Archives/edgar/data/1447028/000144702820000038/FilingSummary.xml


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [21]:
print(xml_all)
print(len(xml_all))


['https://www.sec.gov/Archives/edgar/data/1447028/000144702819000006/FilingSummary.xml', 'https://www.sec.gov/Archives/edgar/data/1070494/000156459019004596/FilingSummary.xml', 'https://www.sec.gov/Archives/edgar/data/1621227/000155837020001571/FilingSummary.xml']
3


In [22]:
master_all = []
master_dic = {}
all_reports = []
for xml in xml_all: 
    master_reports = []
    base_url = xml.replace('FilingSummary.xml', '')

    # request and parse the content
    content = requests.get(xml).content
    soup = BeautifulSoup(content, 'lxml') #format xml

    # find the 'myreports' tag because this contains all the individual reports submitted.
    reports = soup.find('myreports')
    if reports == None:
        pass
    else: 
       

        #I want a list to store all the individual components of the report, so create the master list.
    

        for report in reports.find_all('report')[:-1]: 

                # let's create a dictionary to store all the different parts we need.
            master_dic[base_url] = {}
            master_dic[base_url]['name_short'] = report.shortname.text
            master_dic[base_url]['name_long'] = report.longname.text
            master_dic[base_url]['position'] = report.position.text
            master_dic[base_url]['category'] = report.menucategory.text
            master_dic[base_url]['url'] = base_url + report.htmlfilename.text
#         #OG
            report_dict = {}
            report_dict['name_short'] = report.shortname.text
            report_dict['name_long'] = report.longname.text
            report_dict['position'] = report.position.text
            report_dict['category'] = report.menucategory.text
            report_dict['url'] = base_url + report.htmlfilename.text
        


#         # append the dictionary to the master list.
            master_reports.append(report_dict)
        master_all.append(master_reports)

# #         print('-'*100)
# #         print(base_url + report.htmlfilename.text)
# #         print(report.longname.text)
# #         print(report.shortname.text)
# #         print(report.menucategory.text)
# #         print(report.position.text)



In [191]:
print(len(master_dic))
print(master_dic)

8
{'https://www.sec.gov/Archives/edgar/data/1447028/000144702819000006/': {'name_short': 'Interim Financial Data (Unaudited) (Details)', 'name_long': '2418402 - Disclosure - Interim Financial Data (Unaudited) (Details)', 'position': '85', 'category': 'Details', 'url': 'https://www.sec.gov/Archives/edgar/data/1447028/000144702819000006/R85.htm'}, 'https://www.sec.gov/Archives/edgar/data/1070494/000156459019004596/': {'name_short': 'SCHEDULE II - Valuation and Qualifying Accounts (Detail)', 'name_long': '100530 - Disclosure - SCHEDULE II - Valuation and Qualifying Accounts (Detail)', 'position': '54', 'category': 'Details', 'url': 'https://www.sec.gov/Archives/edgar/data/1070494/000156459019004596/R54.htm'}, 'https://www.sec.gov/Archives/edgar/data/1621227/000155837020001571/': {'name_short': 'Subsequent events (Details)', 'name_long': '41501 - Disclosure - Subsequent events (Details)', 'position': '98', 'category': 'Details', 'url': 'https://www.sec.gov/Archives/edgar/data/1621227/00015

In [23]:
print(master_dic)

{'https://www.sec.gov/Archives/edgar/data/1447028/000144702819000006/': {'name_short': 'Interim Financial Data (Unaudited) (Details)', 'name_long': '2418402 - Disclosure - Interim Financial Data (Unaudited) (Details)', 'position': '85', 'category': 'Details', 'url': 'https://www.sec.gov/Archives/edgar/data/1447028/000144702819000006/R85.htm'}, 'https://www.sec.gov/Archives/edgar/data/1070494/000156459019004596/': {'name_short': 'SCHEDULE II - Valuation and Qualifying Accounts (Detail)', 'name_long': '100530 - Disclosure - SCHEDULE II - Valuation and Qualifying Accounts (Detail)', 'position': '54', 'category': 'Details', 'url': 'https://www.sec.gov/Archives/edgar/data/1070494/000156459019004596/R54.htm'}, 'https://www.sec.gov/Archives/edgar/data/1621227/000155837020001571/': {'name_short': 'Subsequent events (Details)', 'name_long': '41501 - Disclosure - Subsequent events (Details)', 'position': '98', 'category': 'Details', 'url': 'https://www.sec.gov/Archives/edgar/data/1621227/0001558

In [24]:
print(master_dic['https://www.sec.gov/Archives/edgar/data/1447028/000144702819000006/'])

{'name_short': 'Interim Financial Data (Unaudited) (Details)', 'name_long': '2418402 - Disclosure - Interim Financial Data (Unaudited) (Details)', 'position': '85', 'category': 'Details', 'url': 'https://www.sec.gov/Archives/edgar/data/1447028/000144702819000006/R85.htm'}


In [226]:
for company in master_dic.keys():
    dic = master_dic[company]
    print(dic)
#     print('company',company)

#     print(dic['name_short'])
#     print(dic['url'])
     

{'name_short': 'Interim Financial Data (Unaudited) (Details)', 'name_long': '2418402 - Disclosure - Interim Financial Data (Unaudited) (Details)', 'position': '85', 'category': 'Details', 'url': 'https://www.sec.gov/Archives/edgar/data/1447028/000144702819000006/R85.htm'}
{'name_short': 'SCHEDULE II - Valuation and Qualifying Accounts (Detail)', 'name_long': '100530 - Disclosure - SCHEDULE II - Valuation and Qualifying Accounts (Detail)', 'position': '54', 'category': 'Details', 'url': 'https://www.sec.gov/Archives/edgar/data/1070494/000156459019004596/R54.htm'}
{'name_short': 'Subsequent events (Details)', 'name_long': '41501 - Disclosure - Subsequent events (Details)', 'position': '98', 'category': 'Details', 'url': 'https://www.sec.gov/Archives/edgar/data/1621227/000155837020001571/R98.htm'}
{'name_short': 'Uncategorized Items - aeri-1231201910xk.htm', 'name_long': 'Uncategorized Items - aeri-1231201910xk.htm', 'position': '73', 'category': 'Cover', 'url': 'https://www.sec.gov/Archi

In [203]:
count = 0 
for company in master_all:
    print(company)
    for report_dict in company: 
        print(report_dict)

#         # print some info and store it in the statements url.
#             print('-'*100)
#             print(report_dict['name_short'])
#             print(report_dict['url'])

# print(count)



[{'name_short': 'Document and Entity Information', 'name_long': '0001000 - Document - Document and Entity Information', 'position': '1', 'category': 'Cover', 'url': 'https://www.sec.gov/Archives/edgar/data/1447028/000144702819000006/R1.htm'}, {'name_short': 'Consolidated Balance Sheets', 'name_long': '1001000 - Statement - Consolidated Balance Sheets', 'position': '2', 'category': 'Statements', 'url': 'https://www.sec.gov/Archives/edgar/data/1447028/000144702819000006/R2.htm'}, {'name_short': 'Consolidated Balance Sheets (Parenthetical)', 'name_long': '1001501 - Statement - Consolidated Balance Sheets (Parenthetical)', 'position': '3', 'category': 'Statements', 'url': 'https://www.sec.gov/Archives/edgar/data/1447028/000144702819000006/R3.htm'}, {'name_short': 'Consolidated Statements of Operations and Comprehensive Loss', 'name_long': '1002000 - Statement - Consolidated Statements of Operations and Comprehensive Loss', 'position': '4', 'category': 'Statements', 'url': 'https://www.sec.

In [181]:
print(master_all)

[]


In [26]:
# create the list to hold the statement urls
statements_url = []
count = 0 
for company in master_all:
    for report_dict in company: 

#for report_dict in master_reports:
    
#     # define the statements we want to look for.
#     item1 = r"Consolidated Balance Sheets"
#     item2 = r"Consolidated Statements of Comprehensive Income"
#     item3 = r"Consolidated Statements of Cash Flows"
#         item4 = r"Consolidated Statements of Operations"
        item4 = r"Consolidated Statements of Operations and Comprehensive Loss"

        # store them in a list.
        report_list = [item4]
        #print(report_dict['name_short'] )

        #if the short name can be found in the report list.
        if report_dict['name_short'] in report_list:
            

#             # print some info and store it in the statements url.
#             print('-'*100)
            print(report_dict['name_short'])
#             print(report_dict['url'])

            statements_url.append(report_dict['url'])

Consolidated Statements of Operations and Comprehensive Loss


In [27]:
print(statements_url)
#print(len(statements_url))

['https://www.sec.gov/Archives/edgar/data/1447028/000144702819000006/R4.htm']


# Scraping Financial Statements

In [32]:
# let's assume we want all the statements in a single data set.
statements_data = [] #list of dictionaries

# loop through each statement url
for statement in statements_url:

    # define a dictionary that will store the different parts of the statement.
    statement_data = {}
    statement_data['headers'] = []
    statement_data['sections'] = []
    statement_data['data'] = []
    #print(statement)
    
    # request the statement file content
    content = requests.get(statement).content
    report_soup = BeautifulSoup(content, 'html')
    #print(report_soup)

    # find all the rows, figure out what type of row it is, parse the elements, and store in the statement file list.
    for index, row in enumerate(report_soup.table.find_all('tr')):
        #print(index,row)
        
        # first let's get all the elements.
        cols = row.find_all('td')
        print(cols)
        
#         # if it's a regular row and not a section or a table header
#         if (len(row.find_all('th')) == 0 and len(row.find_all('strong')) == 0): 
#             reg_row = [ele.text.strip() for ele in cols]
#             statement_data['data'].append(reg_row)
            
#         # if it's a regular row and a section but not a table header
#         elif (len(row.find_all('th')) == 0 and len(row.find_all('strong')) != 0):
#             sec_row = cols[0].text.strip()
#             statement_data['sections'].append(sec_row)
            
#         # finally if it's not any of those it must be a header
#         elif (len(row.find_all('th')) != 0):            
#             hed_row = [ele.text.strip() for ele in row.find_all('th')]
#             statement_data['headers'].append(hed_row)
            
#         else:            
#             print('We encountered an error.')

#     # append it to the master list.
#     statements_data.append(statement_data)

[]
[]
[<td class="pl" style="border-bottom: 0px;" valign="top"><a class="a" href="javascript:void(0);" onclick="top.Show.showAR( this, 'defref_us-gaap_RevenuesAbstract', window );"><strong>Revenue</strong></a></td>, <td class="text"> <span></span>
</td>, <td class="text"> <span></span>
</td>]
[<td class="pl" style="border-bottom: 0px;" valign="top"><a class="a" href="javascript:void(0);" onclick="top.Show.showAR( this, 'defref_us-gaap_Revenues', window );">Revenue</a></td>, <td class="nump">$ 5,945<span></span>
</td>, <td class="nump">$ 10,700<span></span>
</td>]
[<td class="pl" style="border-bottom: 0px;" valign="top"><a class="a" href="javascript:void(0);" onclick="top.Show.showAR( this, 'defref_us-gaap_CostsAndExpensesAbstract', window );"><strong>Expenses</strong></a></td>, <td class="text"> <span></span>
</td>, <td class="text"> <span></span>
</td>]
[<td class="pl" style="border-bottom: 0px;" valign="top"><a class="a" href="javascript:void(0);" onclick="top.Show.showAR( this, 'def

In [233]:
#print(statements_data)
#print(len(statements_data))
print(statements_data)


[{'headers': [['Consolidated Statements of Operations and Comprehensive Loss - USD ($) $ in Thousands', '12 Months Ended'], ['Dec. 31, 2018', 'Dec. 31, 2017']], 'sections': ['Revenue', 'Expenses', 'Other income (loss)', 'Items applicable to preferred shares', 'Net loss attributable to common shareholders, per share', 'Weighted average number of common shares', 'Other Comprehensive loss'], 'data': [['Revenue', '$ 5,945', '$ 10,700'], ['Research and development', '57,934', '62,676'], ['General and administrative', '16,002', '16,129'], ['Depreciation of property and equipment', '2,181', '2,027'], ['Site consolidation', '4,797', '0'], ['Impairment of intangible assets', '14,811', '40,798'], ['Total expenses', '95,725', '121,630'], ['Loss from operations', '(89,780)', '(110,930)'], ['Interest income', '3,047', '1,538'], ['Interest expense', '(226)', '(261)'], ['Gain on investments', '24,884', '0'], ['Equity investment (loss)', '(5,562)', '0'], ['Foreign exchange gains/(loss)', '(1,003)', '2

In [197]:
print(type(statements_data))


<class 'list'>


In [222]:
print(len(statements_data))
print(statements_data[3]['headers'][0])

19
['Consolidated Statements of Operations - USD ($) shares in Thousands, $ in Thousands', '12 Months Ended']


In [237]:

for i in range(len(statements_data)):
    


    # Grab the proper components
    income_header =  statements_data[3]['headers'][0]
    income_data = statements_data[3]['data']

    # Put the data in a DataFrame
    income_df = pd.DataFrame(income_data)


    # Define the Index column, rename it, and we need to make sure to drop the old column once we reindex.
    income_df.index = income_df[0] #first col

    income_df.index.name = 'Category'
    income_df = income_df.drop(0, axis = 1) #drop col names


    # Get rid of the '$', '(', ')', and convert the '' to NaNs.
    income_df = income_df.replace('[\$,)]','', regex=True )\
                         .replace( '[(]','-', regex=True)\
                         .replace( '', 'NaN', regex=True)


    # # convert string to float
    income_df = income_df.astype(float)

    # # Change the column headers
    income_df.columns = income_header



    # show the df
    print(income_df)


                                                    Consolidated Statements of Operations and Comprehensive Loss - USD ($) $ in Thousands  \
Category                                                                                                                                    
Research and development                                                                      37046.0                                       
General and administrative                                                                     8605.0                                       
Total operating expenses                                                                      45651.0                                       
Loss from operations                                                                         -45651.0                                       
Change in fair value of preferred stock tranche...                                                NaN                                       
Change in fai

# Converting to DF

In [209]:
# Grab the proper components
income_header =  statements_data[2]['headers'][0]
income_data = statements_data[2]['data']

# Put the data in a DataFrame
income_df = pd.DataFrame(income_data)


# Define the Index column, rename it, and we need to make sure to drop the old column once we reindex.
income_df.index = income_df[0] #first col

income_df.index.name = 'Category'
income_df = income_df.drop(0, axis = 1) #drop col names


# Get rid of the '$', '(', ')', and convert the '' to NaNs.
income_df = income_df.replace('[\$,)]','', regex=True )\
                     .replace( '[(]','-', regex=True)\
                     .replace( '', 'NaN', regex=True)


# # convert string to float
income_df = income_df.astype(float)

# # Change the column headers
income_df.columns = income_header



# show the df
income_df

# # drop the data in a CSV file if needed.
# # income_df.to_csv('income_state.csv')

ValueError: Length mismatch: Expected axis has 2 elements, new values have 3 elements

In [126]:
print(income_df.index)

Index(['Net income',
       'Net change in unrealized gains (losses) on cash flow hedges',
       'Total comprehensive income',
       'Cumulative Effect, Period of Adoption, Adjustment | Debt Investment',
       'Total comprehensive income'],
      dtype='object', name='Category')


In [111]:
print(statements_data[1]['data'])
tf = pd.DataFrame(income_data)
print(tf)
display(income_df)

[['Net income', '$ 2,490', '$ 341', '$ 337'], ['Net change in unrealized gains (losses) on cash flow hedges', '17', '8', '(14)'], ['Total comprehensive income', '2,507', '349', '325'], ['Cumulative Effect, Period of Adoption, Adjustment | Debt Investment', '', '', ''], ['Total comprehensive income', '$ 0', '$ 0', '$ 2']]
                                                   0        1      2      3
0                                         Net income  $ 2,490  $ 341  $ 337
1  Net change in unrealized gains (losses) on cas...       17      8   (14)
2                         Total comprehensive income    2,507    349    325
3  Cumulative Effect, Period of Adoption, Adjustm...                       
4                         Total comprehensive income      $ 0    $ 0    $ 2


,0,1,2,3
0,Net income,"$ 2,490",$ 341,$ 337
1,Net change in unrealized gains (losses) on cas...,17,8,(14)
2,Total comprehensive income,"2,507",349,325
3,"Cumulative Effect, Period of Adoption, Adjustm...",,,
4,Total comprehensive income,$ 0,$ 0,$ 2


In [127]:
# Define the Index column, rename it, and we need to make sure to drop the old column once we reindex.
income_df.index = income_df[0]
income_df.index.name = 'Category'
income_df = income_df.drop(0, axis = 1)

# Display
print('-'*100)
print('Before Regex')
print('-'*100)
display(income_df.head())

# # Get rid of the '$', '(', ')', and convert the '' to NaNs.
# income_df = income_df.replace('[\$,)]','', regex=True )\
#                      .replace( '[(]','-', regex=True)\
#                      .replace( '', 'NaN', regex=True)

# # Display
# print('-'*100)
# print('Before type conversion')
# print('-'*100)
# display(income_df.head())

# # everything is a string, so let's convert all the data to a float.
# income_df = income_df.astype(float)

# # Change the column headers
# income_df.columns = income_header

# # Display
# print('-'*100)
# print('Final Product')
# print('-'*100)

# # show the df
# income_df

# # drop the data in a CSV file if need be.
# # income_df.to_csv('income_state.csv')

----------------------------------------------------------------------------------------------------
Before Regex
----------------------------------------------------------------------------------------------------


,1,2,3
Category,,,
Net income,"$ 2,490",$ 341,$ 337
Net change in unrealized gains (losses) on cash flow hedges,17,8,(14)
Total comprehensive income,"2,507",349,325
"Cumulative Effect, Period of Adoption, Adjustment | Debt Investment",,,
Total comprehensive income,$ 0,$ 0,$ 2


In [68]:
base_url = r"https://www.sec.gov/Archives/edgar/data"

# define a company to search (GOLDMAN SACHS), this requires a CIK number that is defined by the SEC.
cik_num = '886982'

# let's get all the filings for Goldman Sachs in a json format.
# Alternative is .html & .xml
filings_url = make_url(base_url, [cik_num, 'index.json'])
print(filings_url)

https://www.sec.gov/Archives/edgar/data/886982/index.json


In [69]:

# let's first make a function that will make the process of building a url easy.
def make_url(base_url , comp):
    
    url = base_url
    
    # add each component to the base url
    for r in comp:
        url = '{}/{}'.format(url, r)
        
    return url

# EXAMPLE
base_url = r"https://www.sec.gov/Archives/edgar/data"
components = ['886982','000156459019011378', '0001564590-19-011378-index-headers.html']
make_url(base_url, components)

'https://www.sec.gov/Archives/edgar/data/886982/000156459019011378/0001564590-19-011378-index-headers.html'

In [70]:
for report_dict in master_reports:
    print(report_dict)



{'name_short': 'Cover Page', 'name_long': '0001001 - Document - Cover Page', 'position': '1', 'category': 'Cover', 'url': 'https://www.sec.gov/Archives/edgar/data/2488/000162828021001185/R1.htm'}
{'name_short': 'Consolidated Statements of Operations', 'name_long': '1001002 - Statement - Consolidated Statements of Operations', 'position': '2', 'category': 'Statements', 'url': 'https://www.sec.gov/Archives/edgar/data/2488/000162828021001185/R2.htm'}
{'name_short': 'Consolidated Statements of Comprehensive Income', 'name_long': '1002003 - Statement - Consolidated Statements of Comprehensive Income', 'position': '3', 'category': 'Statements', 'url': 'https://www.sec.gov/Archives/edgar/data/2488/000162828021001185/R3.htm'}
{'name_short': 'Consolidated Balance Sheets', 'name_long': '1003004 - Statement - Consolidated Balance Sheets', 'position': '4', 'category': 'Statements', 'url': 'https://www.sec.gov/Archives/edgar/data/2488/000162828021001185/R4.htm'}
{'name_short': 'Consolidated Balance